# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[1, 17]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,1,0.153,12,Heartblade,1:17,Artbridge,2,15,town,2,1,1,17
NArea,1,0.339,9,none,1:16,Artbridge,2,11,plain,3,1,1,16
SArea,1,0.56,15,none,1:18,Artbridge,2,22,plain,0,0,1,18
EArea,1,0.338,13,none,2:17,Artbridge,2,11,plain,0,0,2,17
WArea,1,0.27,12,none,0:17,Artbridge,2,19,plain,0,0,0,17


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

True

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
6:0,0,0.0,6,6:0,8.0,desert,NaN,1,Fallwish,0,1,0,0.613
7:0,0,0.0,7,7:0,9.0,desert,NaN,1,Fallwish,0,1,0,0.850
10:0,0,0.0,10,10:0,19.0,desert,NaN,1,Fallwish,0,1,0,0.607
11:0,0,0.0,11,11:0,20.0,desert,NaN,1,Fallwish,0,1,0,0.578
38:0,0,0.0,38,38:0,9.0,desert,NaN,5,Kingsborourgh,0,1,0,0.658


You can also _mask_ the terrain where the character has no knowledge. 

In [7]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,0.0,0,0:0,0.0,ocean,NaN,1,Fallwish,0,1,0,0.221
1:0,0,0.0,1,1:0,2.0,ocean,NaN,1,Fallwish,0,1,0,-0.567
2:0,0,0.0,2,2:0,1.0,ocean,NaN,1,Fallwish,0,1,0,0.406
3:0,0,0.0,3,3:0,0.0,ocean,NaN,1,Fallwish,0,1,0,0.081
4:0,0,0.0,4,4:0,1.0,ocean,NaN,1,Fallwish,0,1,0,-0.416


## Analysis via list comprehension

Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [8]:
world.people


[Stiboldox the Speaker of Fairhail,
 Akillit the Speaker of Heartblade,
 Feffis the Speaker of Buckkeep,
 Hicklab the commoner,
 Chacher the Speaker of Kingsfold,
 Scommaker the Speaker of Stonemark,
 Elekellunt the commoner,
 Oggesh the Speaker of Ratcrest,
 Shoppol the Speaker of Foundersborourgh,
 Ulcklolen the commoner,
 Groklipoch the commoner,
 Tavort the commoner,
 Trihorkol the Speaker of Tearblaze,
 Erelordadle the Speaker of Tearrest,
 Oltoldob the commoner,
 Quiwildem the commoner,
 Urcarraker the Speaker of Foundersplace,
 Mospaker the Speaker of Fentown,
 Sesalkomir the commoner,
 Ekipeltart the commoner,
 Anikilleb the commoner,
 Kilsoth the commoner,
 Chacinnal the Speaker of Snakeblade,
 Domatter the Speaker of Leafspring,
 Febranam the Speaker of Landsblade,
 Saldrad the commoner,
 Erjenna the commoner,
 Fehorkard the commoner,
 Nozac the commoner,
 Quorpal the Speaker of Rockmelt,
 Anascin the Speaker of Godjaw,
 Truladdep the commoner,
 Prankand the commoner,
 Heklip

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [9]:
[[(person.name,person.temperment) for person in t.get_population(world) if person.temperment>.5] for t in world.towns]

[[('Ulcklolen', 0.53)],
 [('Hicklab', 0.98), ('Elekellunt', 1.0)],
 [('Ekipeltart', 0.93),
  ('Saldrad', 0.81),
  ('Truladdep', 0.61),
  ('Quildam', 0.73)],
 [('Chacher', 0.78), ('Olppnan', 0.66), ('Scorpove', 0.65), ('Fockra', 0.72)],
 [('Elohut', 0.84)],
 [('Oggesh', 0.8),
  ('Anikilleb', 0.92),
  ('Mepartaker', 0.97),
  ('Shaklan', 0.84)],
 [('Shoppol', 0.81), ('Derpach', 0.9), ('Zhoblat', 0.58)],
 [('Kilsoth', 0.78)],
 [('Erelordadle', 0.74),
  ('Quiwildem', 0.69),
  ('Lokliport', 0.52),
  ('Thaladdel', 0.62)],
 [('Prankand', 0.55), ('Akiyellox', 0.59)],
 [('Nozac', 0.91)],
 [],
 [('Nohorkah', 0.87)],
 [],
 [],
 [],
 [('Tricksash', 0.94), ('Groplomir', 0.97)],
 [],
 [('Bochith', 0.59)],
 [('Zhawallob', 0.95), ('Ekolla', 0.53), ('Zhocinnish', 0.98)],
 []]

You can also get summary stats on towns. For example, the average temperment per town.

In [10]:
[(t.name,np.mean([person.temperment for person in t.get_population(world)])) for t in world.towns]

[('Fairhail', 0.3325),
 ('Heartblade', 0.618),
 ('Buckkeep', 0.53),
 ('Kingsfold', 0.608),
 ('Stonemark', 0.485),
 ('Ratcrest', 0.756),
 ('Foundersborourgh', 0.528),
 ('Tearblaze', 0.41000000000000003),
 ('Tearrest', 0.562),
 ('Foundersplace', 0.34800000000000003),
 ('Fentown', 0.68),
 ('Snakeblade', 0.08),
 ('Leafspring', 0.59),
 ('Landsblade', 0.5),
 ('Rockmelt', 0.295),
 ('Godjaw', 0.345),
 ('Badgerchild', 0.955),
 ('Demonsfold', 0.41),
 ('Doomhail', 0.59),
 ('Solspring', 0.82),
 ('Splithammer', 0.03)]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [11]:
w.get_people_where_char_has_visited(world)

[{'town': capitol of Heartblade. Location: [1:17]. Founded 1101 </br>In the nation of Artbridge,
  'people': [Akillit the Speaker of Heartblade,
   Hicklab the commoner,
   Elekellunt the commoner,
   Thommel the commoner,
   Elirpich the ruler of Artbridge]}]

In [12]:
r = w.get_people_where_char_has_visited(world)[0]
r

{'town': capitol of Heartblade. Location: [1:17]. Founded 1101 </br>In the nation of Artbridge,
 'people': [Akillit the Speaker of Heartblade,
  Hicklab the commoner,
  Elekellunt the commoner,
  Thommel the commoner,
  Elirpich the ruler of Artbridge]}

In [13]:
r['town'].nation

'Artbridge'

In [14]:
w.get_relationships_node_map(world)

[{'name': 'Heartblade',
  'title': 'capitol of Heartblade. Location',
  'nation': 'Artbridge',
  'population': 5,
  'type': 'capitol',
  'location': '1:17',
  'founded year': 1101,
  'children': [{'name': 'Akillit',
    'role': 'Speaker of Heartblade',
    'temperment': 0.38,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Hicklab',
    'role': 'commoner',
    'temperment': 0.98,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Elekellunt',
    'role': 'commoner',
    'temperment': 1.0,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Thommel',
    'role': 'commoner',
    'temperment': 0.33,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Elirpich',
    'role': 'ruler of Artbridge',
    'temperment': 0.4,
    'loyalty': 1,
    'type': 'person'}]}]

## Getting map data (mapData): 

You can use this to get lists of people who the character knows. 

In [20]:
[(T.nation,T.name) for T in world.towns]

[('Buckfort', 'Fairhail'),
 ('Artbridge', 'Heartblade'),
 ('Buckfort', 'Buckkeep'),
 ('Fallwish', 'Kingsfold'),
 ('Fallwish', 'Stonemark'),
 ('Tearring', 'Ratcrest'),
 ('Hellhail', 'Foundersborourgh'),
 ('Buckfort', 'Tearblaze'),
 ('Starloch', 'Tearrest'),
 ('Fallfold', 'Foundersplace'),
 ('Artbridge', 'Fentown'),
 ('Starloch', 'Snakeblade'),
 ('Fallwish', 'Leafspring'),
 ('Fallwish', 'Landsblade'),
 ('Starloch', 'Rockmelt'),
 ('Starloch', 'Godjaw'),
 ('Buckfort', 'Badgerchild'),
 ('Fallwish', 'Demonsfold'),
 ('Fallfold', 'Doomhail'),
 ('Kingsborourgh', 'Solspring'),
 ('Tearring', 'Splithammer')]

In [22]:
masked_world.loc[world.Character.get_location_key()]

y                            17
rainfall                     15
x                             1
key                        1:17
elevation                    12
terrain                    town
feature              Heartblade
nation number                 2
nation                Artbridge
visited                       1
aware                         1
turn_last_visited             2
danger                    0.153
Name: 1:17, dtype: object

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [23]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
1:17,17,15.0,1,1:17,12.0,town,Heartblade,2,Artbridge,1,1,2,0.153


## Getting the diplomacy and relationships data for the journal page

In [24]:
rel = w.get_people_where_char_has_visited(world)
rel

[{'town': capitol of Heartblade. Location: [1:17]. Founded 1101 </br>In the nation of Artbridge,
  'people': [Akillit the Speaker of Heartblade,
   Hicklab the commoner,
   Elekellunt the commoner,
   Thommel the commoner,
   Elirpich the ruler of Artbridge]}]

In [25]:
w.get_relationships_node_map(world)

[{'name': 'Heartblade',
  'title': 'capitol of Heartblade. Location',
  'nation': 'Artbridge',
  'population': 5,
  'type': 'capitol',
  'location': '1:17',
  'founded year': 1101,
  'children': [{'name': 'Akillit',
    'role': 'Speaker of Heartblade',
    'temperment': 0.38,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Hicklab',
    'role': 'commoner',
    'temperment': 0.98,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Elekellunt',
    'role': 'commoner',
    'temperment': 1.0,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Thommel',
    'role': 'commoner',
    'temperment': 0.33,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Elirpich',
    'role': 'ruler of Artbridge',
    'temperment': 0.4,
    'loyalty': 1,
    'type': 'person'}]}]

In [26]:
hierarchy = [{'nation':i.name,
              'children':i.diplomacy} for i in world.nations]



In [28]:
relationships = w.get_relationships_node_map(world)
relationships

[{'name': 'Heartblade',
  'title': 'capitol of Heartblade. Location',
  'nation': 'Artbridge',
  'population': 5,
  'type': 'capitol',
  'location': '1:17',
  'founded year': 1101,
  'children': [{'name': 'Akillit',
    'role': 'Speaker of Heartblade',
    'temperment': 0.38,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Hicklab',
    'role': 'commoner',
    'temperment': 0.98,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Elekellunt',
    'role': 'commoner',
    'temperment': 1.0,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Thommel',
    'role': 'commoner',
    'temperment': 0.33,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Elirpich',
    'role': 'ruler of Artbridge',
    'temperment': 0.4,
    'loyalty': 1,
    'type': 'person'}]}]

In [29]:
nations = [s['nation'] for s in relationships]
nations

['Artbridge']

In [30]:
[i for i in relationships]

[{'name': 'Heartblade',
  'title': 'capitol of Heartblade. Location',
  'nation': 'Artbridge',
  'population': 5,
  'type': 'capitol',
  'location': '1:17',
  'founded year': 1101,
  'children': [{'name': 'Akillit',
    'role': 'Speaker of Heartblade',
    'temperment': 0.38,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Hicklab',
    'role': 'commoner',
    'temperment': 0.98,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Elekellunt',
    'role': 'commoner',
    'temperment': 1.0,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Thommel',
    'role': 'commoner',
    'temperment': 0.33,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Elirpich',
    'role': 'ruler of Artbridge',
    'temperment': 0.4,
    'loyalty': 1,
    'type': 'person'}]}]

In [31]:
[{'nation':i,'towns':[t for t in relationships if t['nation']==i]} for i in nations]

[{'nation': 'Artbridge',
  'towns': [{'name': 'Heartblade',
    'title': 'capitol of Heartblade. Location',
    'nation': 'Artbridge',
    'population': 5,
    'type': 'capitol',
    'location': '1:17',
    'founded year': 1101,
    'children': [{'name': 'Akillit',
      'role': 'Speaker of Heartblade',
      'temperment': 0.38,
      'loyalty': 0.5,
      'type': 'person'},
     {'name': 'Hicklab',
      'role': 'commoner',
      'temperment': 0.98,
      'loyalty': 0.5,
      'type': 'person'},
     {'name': 'Elekellunt',
      'role': 'commoner',
      'temperment': 1.0,
      'loyalty': 0.5,
      'type': 'person'},
     {'name': 'Thommel',
      'role': 'commoner',
      'temperment': 0.33,
      'loyalty': 0.5,
      'type': 'person'},
     {'name': 'Elirpich',
      'role': 'ruler of Artbridge',
      'temperment': 0.4,
      'loyalty': 1,
      'type': 'person'}]}]}]

In [32]:
[i for i in nations]

['Artbridge']